In [1]:
import tensorflow as tf


In [2]:
X = tf.range(10)

# this dataset is entirely in the ram


dataset = tf.data.Dataset.from_tensor_slices(X)

In [3]:
# chaining transformation -> repeat 3 times; batch of 7

dataset = dataset.repeat(3).batch(7)
for item in dataset:
    print(item)


tf.Tensor([0 1 2 3 4 5 6], shape=(7,), dtype=int32)
tf.Tensor([7 8 9 0 1 2 3], shape=(7,), dtype=int32)
tf.Tensor([4 5 6 7 8 9 0], shape=(7,), dtype=int32)
tf.Tensor([1 2 3 4 5 6 7], shape=(7,), dtype=int32)
tf.Tensor([8 9], shape=(2,), dtype=int32)


In [4]:
# map

dataset = dataset.map(lambda x: x * 2)

for item in dataset:
    print(item)

tf.Tensor([ 0  2  4  6  8 10 12], shape=(7,), dtype=int32)
tf.Tensor([14 16 18  0  2  4  6], shape=(7,), dtype=int32)
tf.Tensor([ 8 10 12 14 16 18  0], shape=(7,), dtype=int32)
tf.Tensor([ 2  4  6  8 10 12 14], shape=(7,), dtype=int32)
tf.Tensor([16 18], shape=(2,), dtype=int32)


In [5]:
# shuffle with buffer -> buffer

dataset = tf.data.Dataset.range(10).repeat(3) # 0 to 9, three times
dataset = dataset.shuffle(buffer_size=5, seed=42).batch(7)
for item in dataset:
    print(item)

tf.Tensor([0 2 3 6 7 9 4], shape=(7,), dtype=int64)
tf.Tensor([5 0 1 1 8 6 5], shape=(7,), dtype=int64)
tf.Tensor([4 8 7 1 2 3 0], shape=(7,), dtype=int64)
tf.Tensor([5 4 2 7 8 9 9], shape=(7,), dtype=int64)
tf.Tensor([3 6], shape=(2,), dtype=int64)


In [ ]:
# filepaths interleaving

n_readers = 5
dataset = filepath_dataset.interleave(
                                        lambda filepath: tf.data.TextLineDataset(filepath).skip(1),
                                        cycle_length=n_readers)


# Preprocessing input features

In [ ]:
# standardization

means = np.mean(X_train, axis=0, keepdims=True)
stds = np.std(X_train, axis=0, keepdims=True)
eps = keras.backend.epsilon()
model = keras.models.Sequential([keras.layers.Lambda(lambda inputs: (inputs - means) / (stds + eps)),

])

In [ ]:
class Standardization(keras.layers.Layer):
    def adapt(self, data_sample):
        self.means_ = np.mean(data_sample, axis=0, keepdims=True)
        self.stds_ = np.std(data_sample, axis=0, keepdims=True)
        _ p ( _ p , , p )
    def call(self, inputs):
        return (inputs - self.means_) / (self.stds_ + keras.backend.epsilon())


### Categorical -> One hot

In [6]:
# vocabulary -> list of all possible categories

vocab = ["<1H OCEAN", "INLAND", "NEAR OCEAN", "NEAR BAY", "ISLAND"]

# tensor with corresponding indices
indices = tf.range(len(vocab), dtype=tf.int64)

# intializer of lookup table
table_init = tf.lookup.KeyValueTensorInitializer(vocab, indices)
# assign to one of the buckets if not in vocab based on hash value
num_oov_buckets = 2
table = tf.lookup.StaticVocabularyTable(table_init, num_oov_buckets)


In [7]:
# generate one hot encodings 

categories = tf.constant(["NEAR BAY", "DESERT", "INLAND", "INLAND"])
cat_indices = table.lookup(categories)
cat_indices
cat_one_hot = tf.one_hot(cat_indices, depth=len(vocab) + num_oov_buckets)

In [8]:
cat_one_hot

<tf.Tensor: shape=(4, 7), dtype=float32, numpy=
array([[0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0.],
       [0., 1., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0.]], dtype=float32)>

# Using Embeddings

### Create embeddings

In [11]:

embedding_dim = 2
# innitialize randomly => Embedding dimension is a hyperparameter
embed_init = tf.random.uniform([len(vocab) + num_oov_buckets,embedding_dim])
embedding_matrix = tf.Variable(embed_init)
embedding_matrix

<tf.Variable 'Variable:0' shape=(7, 2) dtype=float32, numpy=
array([[0.3420329 , 0.4006294 ],
       [0.7949134 , 0.04551959],
       [0.4818431 , 0.00312006],
       [0.33208966, 0.08944726],
       [0.896736  , 0.23530567],
       [0.2511623 , 0.41745567],
       [0.46479917, 0.27548933]], dtype=float32)>

### encode with embeddings

In [12]:
categories = tf.constant(["NEAR BAY", "DESERT", "INLAND", "INLAND"])
cat_indices = table.lookup(categories)
tf.nn.embedding_lookup(embedding_matrix, cat_indices)

<tf.Tensor: shape=(4, 2), dtype=float32, numpy=
array([[0.33208966, 0.08944726],
       [0.2511623 , 0.41745567],
       [0.7949134 , 0.04551959],
       [0.7949134 , 0.04551959]], dtype=float32)>

In [ ]:
regular_inputs = keras.layers.Input(shape=[8])
categories = keras.layers.Input(shape=[], dtype=tf.string)
cat_indices = keras.layers.Lambda(lambda cats: table.lookup(cats))(categories)
cat_embed = keras.layers.Embedding(input_dim=6, output_dim=2)(cat_indices)
encoded_inputs = keras.layers.concatenate([regular_inputs, cat_embed])
outputs = keras.layers.Dense(1)(encoded_inputs)
model = keras.models.Model(inputs=[regular_inputs, categories],
outputs=[outputs])


# Preprocessing pipeline

In [ ]:
normalization = keras.layers.Normalization()
discretization = keras.layers.Discretization([...])
pipeline = keras.layers.PreprocessingStage([normalization,
discretization])
pipeline.adapt(data_sample)


# Tf Transform
If preprocessing is expensive, handlig it before training is better 

In [ ]:
normalization = keras.layers.Normalization()
discretization = keras.layers.Discretization([...])
pipeline = keras.layers.PreprocessingStage([normalization,
discretization])
pipeline.adapt(data_sample)
